In [1]:
import arff
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
from operator import attrgetter, itemgetter
from io import StringIO
from sklearn.model_selection import train_test_split
import nltk
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import tree
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import make_scorer, confusion_matrix,classification_report,precision_recall_fscore_support as score, average_precision_score
from sklearn import metrics
from sklearn.model_selection import cross_validate

## training model
from sklearn.tree import DecisionTreeClassifier

import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV

In [13]:
def get_recal_1(y_true, y_pred):
    precision, recall, fscore, support = score(y_true, y_pred)
    print(classification_report(y_true, y_pred))
    print(recall)
    return(recall[1])

In [3]:
nltk.download('stopwords')
nltk.download('rslp')
stopwords = nltk.corpus.stopwords.words('portuguese')
stemmer = nltk.stem.RSLPStemmer()
nltk.download('punkt')


def clean_text(txt):
    text = ''
    for w in nltk.word_tokenize(txt):
        if w not in stopwords:
            text = text + stemmer.stem(w) + ' '
    return text.strip()

[nltk_data] Downloading package stopwords to /home/shivwa/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package rslp to /home/shivwa/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package punkt to /home/shivwa/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Fase 1

In [4]:
data = arff.load(open('../OffComBR3.arff'))
df = pd.DataFrame(data['data'])
df.columns = ['hate', 'sentence']

# transforming 'yes' into 1 and 'no' into 0
df['hate'] = df['hate'].apply(lambda x: 1 if x == 'yes' else 0)

X = df['sentence'].tolist()
y = df['hate'].tolist()

cl =  Pipeline([
        ('tfidf',TfidfVectorizer()),
        ('clf', DecisionTreeClassifier()),
        ])

scores = cross_validate(cl, X, y,
                        cv=5, return_train_score=True, scoring=make_scorer(get_recal_1))

              precision    recall  f1-score   support

           0       0.85      0.86      0.85       167
           1       0.39      0.37      0.38        41

   micro avg       0.76      0.76      0.76       208
   macro avg       0.62      0.61      0.62       208
weighted avg       0.76      0.76      0.76       208

              precision    recall  f1-score   support

           0       0.87      0.78      0.82       166
           1       0.36      0.51      0.42        41

   micro avg       0.72      0.72      0.72       207
   macro avg       0.61      0.64      0.62       207
weighted avg       0.77      0.72      0.74       207

              precision    recall  f1-score   support

           0       0.85      0.79      0.82       166
           1       0.31      0.40      0.35        40

   micro avg       0.71      0.71      0.71       206
   macro avg       0.58      0.59      0.58       206
weighted avg       0.74      0.71      0.73       206

              preci

## Fase 2

In [5]:
data = arff.load(open('../OffComBR3.arff'))
df = pd.DataFrame(data['data'])
df.columns = ['hate', 'sentence']

# transforming 'yes' into 1 and 'no' into 0
df['hate'] = df['hate'].apply(lambda x: 1 if x == 'yes' else 0)

X = df['sentence'].tolist()
y = df['hate'].tolist()

## Cleaning text before

for i in range(len(X)):
    n_txt = clean_text(X[i])
    X[i] = n_txt
    
cl =  Pipeline([
        ('tfidf',TfidfVectorizer()),
        ('clf', DecisionTreeClassifier()),
        ])

scores = cross_validate(cl, X, y,
                        cv=5, return_train_score=False, scoring=make_scorer(get_recal_1))

              precision    recall  f1-score   support

           0       0.84      0.92      0.88       167
           1       0.48      0.29      0.36        41

   micro avg       0.80      0.80      0.80       208
   macro avg       0.66      0.61      0.62       208
weighted avg       0.77      0.80      0.78       208

              precision    recall  f1-score   support

           0       0.83      0.85      0.84       166
           1       0.34      0.32      0.33        41

   micro avg       0.74      0.74      0.74       207
   macro avg       0.59      0.58      0.59       207
weighted avg       0.74      0.74      0.74       207

              precision    recall  f1-score   support

           0       0.85      0.78      0.81       166
           1       0.31      0.42      0.36        40

   micro avg       0.71      0.71      0.71       206
   macro avg       0.58      0.60      0.59       206
weighted avg       0.75      0.71      0.72       206

              preci

## Fase 3

In [6]:
data = arff.load(open('../OffComBR3.arff'))
df = pd.DataFrame(data['data'])
df.columns = ['hate', 'sentence']

# transforming 'yes' into 1 and 'no' into 0
df['hate'] = df['hate'].apply(lambda x: 1 if x == 'yes' else 0)

X = df['sentence'].tolist()
y = df['hate'].tolist()

## Cleaning text before

for i in range(len(X)):
    n_txt = clean_text(X[i])
    X[i] = n_txt
    
cl =  Pipeline([
        ('tfidf',TfidfVectorizer()),
        ('clf', DecisionTreeClassifier()),
        ])
    
parameters = {'tfidf__ngram_range': [(1,1), (1,2), (1,3), (1,4)],
              'clf__criterion': ('gini', 'entropy'),
              'clf__class_weight': ({0: 1, 1: 1}, {0: 1, 1: 2}, {0: 1, 1: 3}, {0: 1, 1: 4}, {0: 2, 1: 1},
                                    {0: 2, 1: 1}, {0: 3, 1: 1}, {0: 4, 1: 1}),
              'clf__min_samples_split': (2, 3, 4, 5, 6, 7, 8)
             }

gs_clf = GridSearchCV(cl, parameters, cv=5, iid=False, n_jobs=-1, scoring='recall')

gs_clf.fit(X, y)
print(gs_clf.best_score_)
print(gs_clf.best_params_)

0.5795121951219512
{'clf__class_weight': {0: 1, 1: 4}, 'clf__criterion': 'gini', 'clf__min_samples_split': 5, 'tfidf__ngram_range': (1, 2)}


In [18]:
cl =  Pipeline([
        ('tfidf',TfidfVectorizer(ngram_range=(1, 3))),
        ('clf', DecisionTreeClassifier(class_weight={0: 1, 1: 2}, criterion='gini', min_samples_split=3)),
        ])

scores = cross_validate(cl, X, y,
                        cv=5, return_train_score=False, scoring=make_scorer(get_recal_1))

              precision    recall  f1-score   support

           0       0.85      0.74      0.79       167
           1       0.31      0.46      0.37        41

   micro avg       0.69      0.69      0.69       208
   macro avg       0.58      0.60      0.58       208
weighted avg       0.74      0.69      0.71       208

[0.74251497 0.46341463]
              precision    recall  f1-score   support

           0       0.89      0.66      0.76       166
           1       0.33      0.68      0.45        41

   micro avg       0.67      0.67      0.67       207
   macro avg       0.61      0.67      0.60       207
weighted avg       0.78      0.67      0.70       207

[0.6626506  0.68292683]
              precision    recall  f1-score   support

           0       0.89      0.63      0.73       166
           1       0.30      0.68      0.42        40

   micro avg       0.64      0.64      0.64       206
   macro avg       0.60      0.65      0.58       206
weighted avg       0.78   

In [19]:
scores

{'fit_time': array([0.33917713, 0.3157599 , 0.34552932, 0.36317945, 0.35612655]),
 'score_time': array([0.01675677, 0.01406407, 0.01433468, 0.01357198, 0.01354027]),
 'test_score': array([0.46341463, 0.68292683, 0.675     , 0.375     , 0.475     ])}